# 第3章: 正規表現

[正規表現解析ツール](https://regexper.com/#a%7B2%2C5%7Db*)

### memo

wgetのオプションについて
-P : データ格納先を決定
-N : 新しいファイルであればダウンロード

# 20. JSONデータの読み込み
Wikipedia記事のJSONファイルを読み込み，「イギリス」に関する記事本文を表示せよ．問題21-29では，ここで抽出した記事本文に対して実行せよ．

In [1]:
import json,gzip
with gzip.open("data/jawiki-country.json.gz") as f:
    for chunk in f:
        doc = json.loads(chunk)
        if 'イギリス' == doc['title']:
            UK_text = doc['text']
            break
#以降はUK_textを使う

In [2]:
#loadsのsはstringのs

## Shell

In [3]:
#!man jq

In [4]:
!zcat data/jawiki-country.json.gz | grep '"title": "イギリス"' | jq --raw-output .text | head

{{redirect|UK}}
{{基礎情報 国
|略名 = イギリス
|日本語国名 = グレートブリテン及び北アイルランド連合王国
|公式国名 = {{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br/>
*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（[[スコットランド・ゲール語]]）<br/>
*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（[[ウェールズ語]]）<br/>
*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（[[アイルランド語]]）<br/>
*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（[[コーンウォール語]]）<br/>
*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（[[スコットランド語]]）<br/>
Error: writing output failed: Broken pipe


# 21. カテゴリ名を含む行を抽出
記事中でカテゴリ名を宣言している行を抽出せよ．

In [5]:
for line in UK_text.split('\n'):
    if "[[Category:" in line:
        print(line)

[[Category:イギリス|*]]
[[Category:英連邦王国|*]]
[[Category:G8加盟国]]
[[Category:欧州連合加盟国]]
[[Category:海洋国家]]
[[Category:君主国]]
[[Category:島国|くれいとふりてん]]
[[Category:1801年に設立された州・地域]]


In [6]:
for line in UK_text.split('\n'):
    if line.startswith("[[Category:"):
        print(line)

[[Category:イギリス|*]]
[[Category:英連邦王国|*]]
[[Category:G8加盟国]]
[[Category:欧州連合加盟国]]
[[Category:海洋国家]]
[[Category:君主国]]
[[Category:島国|くれいとふりてん]]
[[Category:1801年に設立された州・地域]]


In [7]:
import re

In [8]:
for line in re.finditer(r"\[\[Category:.*\]\]",UK_text):
    print(line.group())

[[Category:イギリス|*]]
[[Category:英連邦王国|*]]
[[Category:G8加盟国]]
[[Category:欧州連合加盟国]]
[[Category:海洋国家]]
[[Category:君主国]]
[[Category:島国|くれいとふりてん]]
[[Category:1801年に設立された州・地域]]


In [9]:
for line in re.compile(r"\[{2}Category.+]{2}").findall(UK_text):
    print(line)

[[Category:イギリス|*]]
[[Category:英連邦王国|*]]
[[Category:G8加盟国]]
[[Category:欧州連合加盟国]]
[[Category:海洋国家]]
[[Category:君主国]]
[[Category:島国|くれいとふりてん]]
[[Category:1801年に設立された州・地域]]


## Shell

In [10]:
!zcat data/jawiki-country.json.gz | grep '"title": "イギリス"' | jq -r .text | grep "Category"

[[Category:イギリス|*]]
[[Category:英連邦王国|*]]
[[Category:G8加盟国]]
[[Category:欧州連合加盟国]]
[[Category:海洋国家]]
[[Category:君主国]]
[[Category:島国|くれいとふりてん]]
[[Category:1801年に設立された州・地域]]


# 22. カテゴリ名の抽出

記事のカテゴリ名を（行単位ではなく名前で）抽出せよ．

In [11]:
for line in re.finditer(r"(?<=Category:)[^\]|]+",UK_text):
    print(line.group(0))

イギリス
英連邦王国
G8加盟国
欧州連合加盟国
海洋国家
君主国
島国
1801年に設立された州・地域


In [12]:
for line in re.finditer(r"\[\[Category:([^\]|]+)",UK_text):
    print(line.group(1))

イギリス
英連邦王国
G8加盟国
欧州連合加盟国
海洋国家
君主国
島国
1801年に設立された州・地域


### memo

#### 後読みアサーション
いま見ている文字の左側(もう読んだ部分)がマッチするか確かめる
#### 先読みアサーション
いま見ている文字の右側(まだ読んでない部分)がマッチするか確かめる

# 23. セクション構造

記事中に含まれるセクション名とそのレベル（例えば"== セクション名 =="なら1）を表示せよ．

In [13]:
for line in re.finditer(r"(={2,})(.+?)={2,}",UK_text):
    print(line.group(2).strip(),line.group(1).count("=")-1)

国名 1
歴史 1
地理 1
気候 2
政治 1
外交と軍事 1
地方行政区分 1
主要都市 2
科学技術 1
経済 1
鉱業 2
農業 2
貿易 2
通貨 2
企業 2
交通 1
道路 2
鉄道 2
海運 2
航空 2
通信 1
国民 1
言語 2
宗教 2
婚姻 2
教育 2
文化 1
食文化 2
文学 2
哲学 2
音楽 2
イギリスのポピュラー音楽 3
映画 2
コメディ 2
国花 2
世界遺産 2
祝祭日 2
スポーツ 1
サッカー 2
競馬 2
モータースポーツ 2
脚注 1
関連項目 1
外部リンク 1


### memo

In [14]:
# 左右対称のものをマッチさせたい時、
test = re.compile(r"(={2,})(.+?)\1")

#名前をつける時
test2 = re.compile(r"(?P<equal>={2,5})([^=]*?)")

# 24. ファイル参照の抽出
記事から参照されているメディアファイルをすべて抜き出せ．

In [15]:
for line in re.finditer(r"(?<=File:|ファイル:)[^\]|]+",UK_text):
    print(line.group())

Royal Coat of Arms of the United Kingdom.svg
Battle of Waterloo 1815.PNG
The British Empire.png
Uk topo en.jpg
BenNevis2005.jpg
Elizabeth II greets NASA GSFC employees, May 8, 2007 edit.jpg
Palace of Westminster, London - Feb 2007.jpg
David Cameron and Barack Obama at the G20 Summit in Toronto.jpg
Soldiers Trooping the Colour, 16th June 2007.jpg
Scotland Parliament Holyrood.jpg
London.bankofengland.arp.jpg
City of London skyline from London City Hall - Oct 2008.jpg
Oil platform in the North SeaPros.jpg
Eurostar at St Pancras Jan 2008.jpg
Heathrow T5.jpg
Anglospeak.svg
CHANDOS3.jpg
The Fabs.JPG
PalaceOfWestminsterAtNight.jpg
Westminster Abbey - West Door.jpg
Edinburgh Cockburn St dsc06789.jpg
Canterbury Cathedral - Portal Nave Cross-spire.jpeg
Kew Gardens Palm House, London - July 2009.jpg
2005-06-27 - United Kingdom - England - London - Greenwich.jpg
Stonehenge2007 07 30.jpg
Yard2.jpg
Durham Kathedrale Nahaufnahme.jpg
Roman Baths in Bath Spa, England - July 2006.jpg
Fountains Abbey vie

# 25. テンプレートの抽出

記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し，辞書オブジェクトとして格納せよ．

In [16]:
text = re.search(r"{{基礎情報 国\n\|(.*?)\n}}",UK_text,re.S).group(1)
dic = {line.split(' = ')[0] : line.split(' = ')[1] for line in text.split('\n|')}

dic

{'略名': 'イギリス',
 '日本語国名': 'グレートブリテン及び北アイルランド連合王国',
 '公式国名': '{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br/>\n*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（[[スコットランド・ゲール語]]）<br/>\n*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（[[ウェールズ語]]）<br/>\n*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（[[アイルランド語]]）<br/>\n*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（[[コーンウォール語]]）<br/>\n*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（[[スコットランド語]]）<br/>\n**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>',
 '国旗画像': 'Flag of the United Kingdom.svg',
 '国章画像': '[[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]',
 '国章リンク': '（[[イギリスの国章|国章]]）',
 '標語': '{{lang|fr|Dieu et mon droit}}<br/>（[[フランス語]]:神と私の権利）',
 '国歌': '[[女王陛下万歳|神よ女王陛下を守り給え]]',
 '位置画像'

### memo
- itertools
    - takewhile
    - dropwhile
    - groupby

# 26. 強調マークアップの除去

25の処理時に，テンプレートの値からMediaWikiの強調マークアップ（弱い強調，強調，強い強調のすべて）を除去してテキストに変換せよ（参考: マークアップ早見表）．

- ''弱い強調''
- '''強調'''
- '''''強い強調'''''

In [17]:
def el_emphasis(string):
    return re.sub(r"'{2,5}","",string)

In [18]:
test = "最初に''弱い強調''、次に'''強調'''、最後に'''''強い強調'''''をする"
print(test)
print(el_emphasis(test))

最初に''弱い強調''、次に'''強調'''、最後に'''''強い強調'''''をする
最初に弱い強調、次に強調、最後に強い強調をする


In [19]:
text = re.search(r"{{基礎情報 国\n\|(.*?)\n}}",UK_text,re.S).group(1)
dic = {line.split(' = ')[0] : el_emphasis(line.split(' = ')[1]) for line in text.split('\n|')}

dic

{'略名': 'イギリス',
 '日本語国名': 'グレートブリテン及び北アイルランド連合王国',
 '公式国名': '{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br/>\n*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（[[スコットランド・ゲール語]]）<br/>\n*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（[[ウェールズ語]]）<br/>\n*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（[[アイルランド語]]）<br/>\n*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（[[コーンウォール語]]）<br/>\n*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（[[スコットランド語]]）<br/>\n**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>',
 '国旗画像': 'Flag of the United Kingdom.svg',
 '国章画像': '[[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]',
 '国章リンク': '（[[イギリスの国章|国章]]）',
 '標語': '{{lang|fr|Dieu et mon droit}}<br/>（[[フランス語]]:神と私の権利）',
 '国歌': '[[女王陛下万歳|神よ女王陛下を守り給え]]',
 '位置画像'

# 27. 内部リンクの除去

26の処理に加えて，テンプレートの値からMediaWikiの内部リンクマークアップを除去し，テキストに変換せよ（参考: マークアップ早見表）．

- [[記事名]]
- [[記事名|表示文字]]
- [[記事名#節名|表示文字]]

In [20]:
def el_internal_link(string):
    tmp = re.sub(r"\[\[[^\]]*\|(.*?)\]\]",r"\1",string)
    return re.sub(r"\[\[(.*?)]]",r"\1",tmp)

In [21]:
test = "[[記事名]],[[記事名|表示文字]],[[記事名#節名|表示文字]],[[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]"
print(test)
print(el_internal_link(test))

[[記事名]],[[記事名|表示文字]],[[記事名#節名|表示文字]],[[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]
記事名,表示文字,表示文字,イギリスの国章


In [22]:
text = re.search(r"{{基礎情報 国\n\|(.*?)\n}}",UK_text,re.S).group(1)
dic = {line.split(' = ')[0] : el_emphasis(el_internal_link(line.split(' = ')[1])) for line in text.split('\n|')}

dic

{'略名': 'イギリス',
 '日本語国名': 'グレートブリテン及び北アイルランド連合王国',
 '公式国名': '{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br/>\n*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（スコットランド・ゲール語）<br/>\n*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（ウェールズ語）<br/>\n*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（アイルランド語）<br/>\n*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（コーンウォール語）<br/>\n*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（スコットランド語）<br/>\n**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>',
 '国旗画像': 'Flag of the United Kingdom.svg',
 '国章画像': 'イギリスの国章',
 '国章リンク': '（国章）',
 '標語': '{{lang|fr|Dieu et mon droit}}<br/>（フランス語:神と私の権利）',
 '国歌': '神よ女王陛下を守り給え',
 '位置画像': 'Location_UK_EU_Europe_001.svg',
 '公用語': '英語（事実上）',
 '首都': 'ロンドン',
 '最大都市': 'ロンドン',
 '元首等肩書': '女王',
 '元首

### memo

In [23]:
# 関数をre.subに使うこともできる
# ex)
#     re.sub(regex,f,text)
#     関数fの引数はマッチオブジェクト

# 28. MediaWikiマークアップの除去

27の処理に加えて，テンプレートの値からMediaWikiマークアップを可能な限り除去し，国の基本情報を整形せよ．
- br
- ref
- <>
- [http: ~ ]
- {{ | | }}

In [24]:
def el_br(string):
    return re.sub(r"<br ?/>","\n",string)

def el_ref(string):
     return re.sub(r"<ref.*?/>","",string)

def el_tab(string):
    return re.sub(r"<.*?>","",string)

def el_http(string):
    return re.sub(r"\[http:.*?\]","",string)

def el_pipe(string):
    return re.sub(r"{{.*?\|.*?\|(.*?)}}",r"\1",string)

In [25]:
el_list = [el_br,el_ref,el_tab,el_internal_link,el_http,el_emphasis,el_pipe]

def el_all(string):
    for el in el_list:
        string = el(string)
    return string

In [26]:
text = re.search(r"{{基礎情報 国\n\|(.*?)\n}}",UK_text,re.S).group(1)
UK_dic = {line.split(' = ')[0] : el_all(line.split(' = ')[1]) for line in text.split('\n|')}

UK_dic

{'略名': 'イギリス',
 '日本語国名': 'グレートブリテン及び北アイルランド連合王国',
 '公式国名': 'United Kingdom of Great Britain and Northern Ireland英語以外での正式国名:\n\n*An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath（スコットランド・ゲール語）\n\n*Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon（ウェールズ語）\n\n*Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann（アイルランド語）\n\n*An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh（コーンウォール語）\n\n*Unitit Kinrick o Great Breetain an Northren Ireland（スコットランド語）\n\n**Claught Kängrick o Docht Brätain an Norlin Airlann、Unitet Kängdom o Great Brittain an Norlin Airlann（アルスター・スコットランド語）',
 '国旗画像': 'Flag of the United Kingdom.svg',
 '国章画像': 'イギリスの国章',
 '国章リンク': '（国章）',
 '標語': 'Dieu et mon droit\n（フランス語:神と私の権利）',
 '国歌': '神よ女王陛下を守り給え',
 '位置画像': 'Location_UK_EU_Europe_001.svg',
 '公用語': '英語（事実上）',
 '首都': 'ロンドン',
 '最大都市': 'ロンドン',
 '元首等肩書': '女王',
 '元首等氏名': 'エリザベス2世',
 '首相等肩書': '首相',
 '首相等氏名': 'デーヴィッド・キャメロン',
 '面積順位': '76',
 '面積大きさ': '1 E11',
 '面積値': '244,820',
 '水面積率': '1.3%',
 '人口統計年': '201

# 29. 国旗画像のURLを取得する

テンプレートの内容を利用し，国旗画像のURLを取得せよ．（ヒント: [MediaWiki API](https://www.mediawiki.org/wiki/API:Main_page/ja)の[imageinfo](https://www.mediawiki.org/wiki/API:Properties/ja#imageinfo_.2F_ii)を呼び出して，ファイル参照をURLに変換すればよい）

### requestsとurllib
requestsのほうが使いやすそう

In [46]:
import requests,urllib

endpoint = 'https://en.wikipedia.org/w/api.php'
payload = {'format' : 'json',
           'action' : 'query',
           'prop' : 'imageinfo',
           'iiprop' : 'url',
           'titles' : f"File:{UK_dic['国旗画像']}"
          }

#requests
r = requests.get(endpoint,payload)
print(re.search(r'"url":"(.*?)"',r.text).group(1))

# #urllib
r = urllib.request.urlopen(f"{endpoint}?{urllib.parse.urlencode(payload)}")
print(re.search(rb'"url":"(.*?)"',r.read()).group(1))

https://upload.wikimedia.org/wikipedia/en/a/ae/Flag_of_the_United_Kingdom.svg
b'https://upload.wikimedia.org/wikipedia/en/a/ae/Flag_of_the_United_Kingdom.svg'


### memo
- iPython image
- mark down
- エラー処理もあり

![](https://upload.wikimedia.org/wikipedia/en/a/ae/Flag_of_the_United_Kingdom.svg)